In [1]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

## Network

In [2]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()

        self.embedding_layer = nn.Embedding(num_embeddings=1000, embedding_dim=6)
        self.rnn = nn.RNN(input_size=6, hidden_size=4, num_layers=1, batch_first=True, bidirectional=True)
        self.proj = nn.Linear(8, 2)

    def forward(self, x, mask=None):
        """
        前向过程
        :param x: [N,T] LongTensor token id列表
        :param mask: [N,T] 实际值位置为1，填充值为0
        :return: 置信度 [N,2]
        """
        # 1. embedding的向量提取
        z1 = self.embedding_layer(x)  # [N,T] -> [N,T,embedding_dim]
        
        # 2. rnn的特征提取
        if mask is None:
            output, _ = self.rnn(z1)  # [N,T,embedding_dim] --> [N,T,hidden_size]
        else:
            lengths = torch.sum(mask, dim=1).to(torch.long)
            # pack_padded_sequence: 从z1中仅提取有效数据
            z2: rnn.PackedSequence = pack_padded_sequence(z1, lengths, batch_first=True, enforce_sorted=False)
            z2, _ = self.rnn(z2)
            output, _ = pad_packed_sequence(z2, batch_first=True)
        
        # 3. 将T个时刻的向量合并成一个向量
        """
        期望：用一个向量来表示整个文本的特征信息，并不需要针对每个时刻的token进行特征向量描述 --> 如何提取一个向量，并且这个向量可以体现当前这个文本。
        方式一: 直接提取最后一个时刻的输出特征向量： output = output[:,-1,:]
        方式二：直接将所有时刻的特征向量求均值: 
            output = torch.mean(output, dim=1) # [N,T,hidden_size] -> [N,hidden_size]
            # 将output里面的值看出每个时刻的输出值:hi
            output = alpha_1*h_1 + alpha_2*h_2 + alpha_3*h_3 +....+alpha_t*h_t
            起始torch.mean操作中，alpha_i等于1/t, t==T
        """
        output = torch.mean(output, dim=1)  # [N,T,hidden_size] -> [N,hidden_size]

        # 4. 决策输出
        scores = self.proj(output)
        return scores


## not mast

In [3]:
net = Network()
_x = torch.randint(100, size=(7, 5))
scores = net(_x)
print(scores)
print(scores.shape)

tensor([[ 0.0626, -0.0744],
        [ 0.0156,  0.1935],
        [-0.0526, -0.1035],
        [-0.2965,  0.1919],
        [-0.1860,  0.0129],
        [-0.0921, -0.2721],
        [-0.0633, -0.0723]], grad_fn=<AddmmBackward0>)
torch.Size([7, 2])


In [4]:
y_pred = torch.argmax(scores, dim=1)
y_pred

tensor([0, 1, 0, 1, 1, 0, 0])

## mask

In [5]:
net = Network()
_x = torch.tensor([
    [2, 3, 5, 4, 6, 2],  # 样本1实际长度为6，填充后大小为6
    [2, 3, 5, 65, 0, 0],  # 样本2实际长度为4，填充后大小为6
    [1, 3, 5, 0, 0, 0]  # 样本3实际长度为3，填充后大小为6
])
_mask = torch.tensor([
    [1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
    [1.0, 1.0, 1.0, 1.0, 0.0, 0.0],
    [1.0, 1.0, 1.0, 0.0, 0.0, 0.0]
])
scores = net(_x, _mask)
print(scores)
print(scores.shape)

tensor([[-0.4025, -0.1584],
        [-0.2626, -0.2636],
        [-0.1955, -0.1135]], grad_fn=<AddmmBackward0>)
torch.Size([3, 2])


In [ ]:
# 假设有一批序列数据，已经按长度降序排列
sequences = [torch.tensor([5,6,7]), torch.tensor([1,2,3,4]), torch.tensor([8,9])]
# 使用pad_sequence进行填充
padded_sequences = pad_sequence(sequences, batch_first=True)


# 计算每个序列的实际长度
lengths = torch.sum(padded_sequences != 0, dim=1)

batch_size, seq_len = padded_sequences.size()
# mask = torch.zeros(batch_size, seq_len, dtype=torch.float)
# for i, length in enumerate(lengths):
#     mask[i, :length] = 1


# 矩阵的广播机制
mask = (torch.arange(seq_len).unsqueeze(0) < lengths.unsqueeze(1)).float()

scores = net(padded_sequences, mask)
print(scores)
print(scores.shape)

In [11]:
padded_sequences

tensor([[1, 2, 3, 4],
        [5, 6, 7, 0],
        [8, 9, 0, 0]])

## RNN输入数据的时间T为什么可以不一样？

RNN的参数结构

## pack_padded_sequence与pad_packed_sequence

In [7]:
lengths.sort(descending=True)

torch.return_types.sort(
values=tensor([4, 3, 2]),
indices=tensor([1, 0, 2]))

In [9]:
# 压缩填充序列
packed_input = pack_padded_sequence(padded_sequences, lengths, batch_first=True, enforce_sorted=False)
packed_input

PackedSequence(data=tensor([1, 5, 8, 2, 6, 9, 3, 7, 4]), batch_sizes=tensor([3, 3, 2, 1]), sorted_indices=tensor([1, 0, 2]), unsorted_indices=tensor([1, 0, 2]))

enforce_sorted 参数在 pack_padded_sequence 函数中用于指定输入序列是否已经按长度降序排列。

- enforce_sorted=True：（默认值）这意味着输入序列必须按长度降序排列。如果设置为 True，函数会假设输入序列已经按照长度从长到短排序，这可以提高处理效率，因为内部实现可以利用这种排序来优化计算。如果你的数据没有预先排序，使用这个选项可能会导致错误的结果。

- enforce_sorted=False：这意味着输入序列不必按长度降序排列。如果设置为 False，函数会自动处理未排序的序列，这通常涉及到在内部对序列进行排序和后续的反排序操作。这提供了便利，因为你不需要在调用 pack_padded_sequence 之前手动对序列进行排序，但可能会稍微降低性能，因为需要额外的排序步骤。

In [9]:
# 解压缩序列
unpacked_output, output_lengths = pad_packed_sequence(packed_input, batch_first=True)

print(unpacked_output)
print(output_lengths)

tensor([[1, 2, 3, 4],
        [5, 6, 7, 0],
        [8, 9, 0, 0]])
tensor([4, 3, 2])


## 稠密矩阵与稀疏矩阵

In [10]:
import torch

# 创建一个稠密矩阵
dense_matrix = torch.tensor([[1, 0, 0], [0, 23, 0], [0, 0, 5], [0, 0, 56]], dtype=torch.float32)
print("Dense Matrix:")
print(dense_matrix)

# 转换为稀疏矩阵
sparse_matrix = dense_matrix.to_sparse()
print("\nSparse Matrix:")
print(sparse_matrix)

# 转换回稠密矩阵
dense_matrix_reconstructed = sparse_matrix.to_dense()
print("\nReconstructed Dense Matrix:")
print(dense_matrix_reconstructed)

Dense Matrix:
tensor([[ 1.,  0.,  0.],
        [ 0., 23.,  0.],
        [ 0.,  0.,  5.],
        [ 0.,  0., 56.]])

Sparse Matrix:
tensor(indices=tensor([[0, 1, 2, 3],
                       [0, 1, 2, 2]]),
       values=tensor([ 1., 23.,  5., 56.]),
       size=(4, 3), nnz=4, layout=torch.sparse_coo)

Reconstructed Dense Matrix:
tensor([[ 1.,  0.,  0.],
        [ 0., 23.,  0.],
        [ 0.,  0.,  5.],
        [ 0.,  0., 56.]])
